# 제4장 Python에 의한 음성 신호 처리

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/r9y9/ttslearn/blob/master/notebooks/ch04_Python-SP.ipynb)

## 4.1 준비

### Python version

In [ ]:
!python -VV

### ttslearn 설치

In [ ]:
%%capture
try:
    import ttslearn
except ImportError:
    !pip install ttslearn

In [ ]:
import ttslearn
ttslearn.__version__

### ttslearn 동작 확인

In [ ]:
from ttslearn.dnntts import DNNTTS
from IPython.display import Audio

engine = DNNTTS()
wav, sr = engine.tts("日本語音声合成のデモです。")
Audio(wav, rate=sr)

In [ ]:
import librosa.display
import matplotlib.pyplot as plt
import numpy as np

fig, ax = plt.subplots(figsize=(8,2))
librosa.display.waveshow(wav.astype(np.float32), sr, ax=ax)
ax.set_xlabel("Time [sec]")
ax.set_ylabel("Amplitude")
plt.tight_layout()

### 패키지 임포트

In [ ]:
%pylab inline
%load_ext autoreload
%autoreload
import IPython
from IPython.display import Audio
import os

In [ ]:
# 시드 고정
from ttslearn.util import init_seed
init_seed(1234)

### 그래프 그리기 설정 (描画周りの設定) // 번역 수정 필요

In [ ]:
from ttslearn.notebook import get_cmap, init_plot_style, savefig
cmap = get_cmap()
init_plot_style()

## 4.2 수치 계산을 위한 Python 라이브러리

### NumPy와 Torch를 이용한 배열 작성

In [ ]:
import numpy as np
import torch

In [ ]:
x = np.zeros((2,2), dtype=np.float32)
x

In [ ]:
y = torch.zeros(2,2, dtype=torch.float)
y

In [ ]:
type(x)

In [ ]:
type(y)

### numpy.ndarray와 torch.Tensor의 인터페이스 차이

In [ ]:
# Numpy에서는 배열 크기를 tuple로 줍니다.
x = np.zeros((1,2,3), dtype=np.float32)
x

In [ ]:
# PyTorch에서는 배열 크기를 다른 인수로 제공합니다.
y = torch.zeros(1, 2, 3, dtype=torch.float32)
y

In [ ]:
x.shape == y.shape

### numpy.ndarray와 torch.Tensor의 상호 변환

In [ ]:
x = np.zeros((2,2), dtype=np.float32)

In [ ]:
y = torch.zeros((2,2), dtype=torch.float32)

In [ ]:
# torch.Tensor에서 numpy.ndarray로의 변환
type(y.numpy())

In [ ]:
# numpy.ndarray에서 torch.Tensor로의 변환
type(torch.from_numpy(x))

### numpy.ndarray와 torch.Tensor의 메모리 공유

In [ ]:
x = np.zeros((2,2), dtype=np.float32)
x

In [ ]:
y = torch.from_numpy(x)
y

In [ ]:
x[0,0] = 1.0 # 메모리가 공유되어 x로의 변경은 y에도 반영됩니다.

In [ ]:
x

In [ ]:
y

## 4.3 음성 파일 가져오기

### scipy.io.wavfile을 이용한 음성 파일 가져오기

In [ ]:
from scipy.io import wavfile
import ttslearn

In [ ]:
sr, wav = wavfile.read(ttslearn.util.example_audio_file())

In [ ]:
sr

In [ ]:
wav.shape

In [ ]:
len(wav) / sr

In [ ]:
wav

In [ ]:
type(wav)

### 음성의 가시화(시각화)

In [ ]:
import librosa.display
import matplotlib.pyplot as plt

# 음성 파일 읽기
sr, x = wavfile.read(ttslearn.util.example_audio_file())

fig, ax = plt.subplots(figsize=(8,2))
librosa.display.waveshow(x.astype(np.float32), sr, ax=ax)

ax.set_xlabel("Time [sec]")
ax.set_ylabel("Amplitude")
plt.tight_layout()

# 그림 4-2
savefig("fig/pyssp_waveshow")

# 오디오 플레이어의 표시
Audio(x.astype(np.float32), rate=sr)

## 4.4 음성의 푸리에 변환

In [ ]:
# 음성 파일 읽기
sr, x = wavfile.read(ttslearn.util.example_audio_file())
# 진폭 스펙트로그램
X = np.abs(np.fft.rfft(x))
# 로그 진폭 스펙트로그램
logX = 20*np.log10(X)

fig, ax = plt.subplots(1, 2, figsize=(10, 4), sharex=True)
freq = np.arange(len(X)) / 2 / len(X) * sr
ax[0].plot(freq, X)
ax[0].set_title("Amplitude spectrum")
ax[0].set_xlim(0, sr // 2)
ax[0].set_xlabel("Frequency [Hz]")
ax[0].set_ylabel("Amplitude")

ax[1].plot(freq, logX)
ax[1].set_title("Log amplitude spectrum")
ax[1].set_xlabel("Frequency [Hz]")
ax[1].set_ylabel("Amplitude [dB]")
plt.tight_layout()

# 그림 4-3
savefig("fig/pyssp_rfftplot")

## 4.5 음성의 단시간 푸리에 변환(STFT, Short Time Fourier Transform)과 그 역변환

### 창함수

In [ ]:
N = 1024
n = np.arange(N)
w = 0.5 - 0.5 * np.cos(2*np.pi * n / N)

fig, ax = plt.subplots(figsize=(6,4))
ax.plot(w)
ax.set_xlim(0, N)
ax.set_xlabel("Time [sample]")
ax.set_ylabel("Amplitude")
plt.tight_layout()

### 단시간 푸리에 변환(STFT) 구현

In [ ]:
def hanning(N):
    n = np.arange(N)
    w = 0.5 - 0.5 * np.cos(2*np.pi * n / N)
    return w

def stft(x, N, S):
    # 창함수(간단하기 때문에 창폭과 프레임길이 N은 같습니다)
    w = hanning(N)
    # 단시간 푸리에 변환 프레임 수
    M = (len(x) - N) // S + 1
    # 단시간 푸리에 변환 결과 저장을 위한 2차원 배열
    X = np.zeros((M, N//2 + 1), dtype=complex)
    # 음성을 어긋나게 잘라내고 푸리에 변환
    for m in range(M):
        x_m = w * x[m*S:m*S+N]
        X[m, :] = np.fft.rfft(x_m)
    return X

### 단시간 푸리에 변환 결과 가시화(시각화)

In [ ]:
# 음성 파일 읽기
sr, x = wavfile.read(ttslearn.util.example_audio_file())

# 5밀리초의 프레임 시프트를 연산합니다.
frame_shift = int(sr * 0.005)
n_fft = 2048
# 스펙트로그램
X = stft(x.astype(np.float32), n_fft, frame_shift)
# 로그 진폭으로 변환
logX = librosa.amplitude_to_db(np.abs(X), ref=np.max)

fig, ax = plt.subplots(1, 1, figsize=(8,4), sharex=True)
img = librosa.display.specshow(logX.T, hop_length=frame_shift, sr=sr, cmap=cmap, x_axis="time", y_axis="hz", ax=ax)
fig.colorbar(img, ax=ax, format="%+2.f dB")
# 음성 출력은 저역에 집중하기 때문에 8000Hz까지 표시한다
ax.set_ylim(0, 8000)

ax.set_xlabel("Time [sec]")
ax.set_ylabel("Frequency [Hz]")
plt.tight_layout()

# 그림 4-5
savefig("fig/pyssp_stft_example")

### librosa.stft를 이용한 단시간 푸리에 변환

librosa.stft 는 STFT 를 실행하기 전에 디폴트로 신호의 첫머리와 끝에 패딩 처리를 합니다. 앞에서 설명한 STFT 구현은 이 처리를 지원하지 않기 때문에 동등한 STFT 결과를 얻기 위해서는 center=False로 패딩 처리를 하지 않도록 설정합니다.

In [ ]:
import librosa

# n_fft: 2048, frame_shift: 240
X = librosa.stft(x.astype(np.float32), n_fft=n_fft, win_length=n_fft, hop_length=frame_shift, window="hann", center=False).T
# 로그 진폭으로 변환
logX = librosa.amplitude_to_db(np.abs(X), ref=np.max)

fig, ax = plt.subplots(1, 1, figsize=(8,4), sharex=True)
img = librosa.display.specshow(logX.T, hop_length=frame_shift, sr=sr, cmap=cmap, x_axis="time", y_axis="hz", ax=ax)
fig.colorbar(img, ax=ax, format="%+2.f dB")
# 음성 출력은 저역에 집중하기 때문에 8000Hz까지 표시한다
ax.set_ylim(0, 8000)

ax.set_xlabel("Time [sec]")
ax.set_ylabel("Frequency [Hz]")
plt.tight_layout()

### 시간 해상도와 주파수 해상도의 트레이드 오프

In [ ]:
def next_power_of_2(x):
    return 1 if x == 0 else 2**(x - 1).bit_length()

fig, ax = plt.subplots(1, 3, figsize=(10,5), sharex=True, sharey=True)

for idx, win_length_ms in enumerate([0.05, 0.02, 0.01]):
    win_length = int(sr * win_length_ms)
    frame_shift = win_length // 4
    n_fft = next_power_of_2(win_length)
    
    X = librosa.stft(x.astype(np.float32), n_fft=n_fft, win_length=n_fft, hop_length=frame_shift).T
    logX =  librosa.amplitude_to_db(np.abs(X), ref=np.max)
    mesh = librosa.display.specshow(
        logX.T, hop_length=frame_shift, sr=sr, cmap=cmap, x_axis="time", y_axis="hz", ax=ax[idx])
    fig.colorbar(mesh, ax=ax[idx], format="%+2.f dB")
    ax[idx].set_title(f"win_length: {win_length}")
    mesh.set_clim(-80, 0)
    ax[idx].set_xlim(1.0, 1.5)
    ax[idx].set_xticks([1.0, 1.25, 1.5])
    # 나중에 라벨을 다시 붙일 테니 여기에서는 지워둔다
    ax[idx].set_ylabel("")

ax[0].set_ylabel("Frequency [Hz]")
for a in ax:
    a.set_xlabel("Time [sec]")
    a.set_ylim(0, 8000)
    a.xaxis.set_major_formatter(FormatStrFormatter('%.1f'))
plt.tight_layout()

# 그림 4-6
savefig("fig/pyssp_stft_tradeoff")

### 역단시간 푸리에 변환(ISTFT, Inverse Short-Time Fourier Transform)을 통한 음성 복원

In [ ]:
# 음성 파일 읽기
sr, x = wavfile.read(ttslearn.util.example_audio_file())
# 5밀리초의 프레임 시프트를 연산합니다.
frame_shift = int(sr * 0.005)
n_fft = 2048

# STFT
X = librosa.stft(x.astype(np.float32), n_fft=n_fft, win_length=n_fft, hop_length=frame_shift, window="hann")
# ISTFT
x_hat = librosa.istft(X, win_length=n_fft, hop_length=frame_shift, window="hann")

IPython.display.display(Audio(x.astype(np.float32), rate=sr))
IPython.display.display(Audio(x_hat.astype(np.float32), rate=sr))

fig, ax = plt.subplots(2, 1, figsize=(8,4), sharey=True)
ax[0].set_title("Original speech")
ax[1].set_title("Reconstructed speech by ISTFT")
librosa.display.waveshow(x.astype(np.float32), sr, ax=ax[0])
librosa.display.waveshow(x_hat.astype(np.float32), sr, ax=ax[1])

for a in ax:
    a.set_xlabel("Time [sec]")
    a.set_ylabel("Amplitude")
plt.tight_layout()

## 4.6 멜스펙트로그램 (mel-spectrogram)

### 멜 필터 뱅크

In [ ]:
sr = 16000
n_fft = 2048
n_mels = 8

# 음성 파일 읽기
sr, x = wavfile.read(ttslearn.util.example_audio_file())
x = x.astype(np.float32)
x = librosa.resample(x, sr, 16000)
sr = 16000

# 5밀리초의 프레임 시프트를 연산합니다.
frame_shift = int(sr * 0.005)
# STFT
X = librosa.stft(x, n_fft=n_fft, win_length=n_fft, hop_length=frame_shift, window="hann")
# 1 프레임을 잘라내다.
X_m = np.abs(X[:, 280])

# 멜 필터 뱅크: n_mels 개의 필터로 구성됩니다.
melfb = librosa.filters.mel(sr, n_fft, n_mels=n_mels, norm=None)
freq = librosa.fft_frequencies(sr, n_fft)

# 멜 필터 뱅크를 표시
fig, ax = plt.subplots(n_mels+1, 2, figsize=(8,10), sharex=True)
ax[0][0].plot(freq, np.ones_like(freq))
ax[0][0].set_title("All pass filter")
ax[0][0].set_ylim(0,1.1)
ax[0][1].plot(freq, X_m)
ax[0][1].set_title("Input amplitude spectrum")
for idx, fb in enumerate(melfb):
    ax[idx+1][0].plot(freq, fb)
    ax[idx+1][0].set_title(f"Filter {idx+1}")
    ax[idx+1][1].plot(freq, fb * X_m)
    ax[idx+1][1].set_title(f"Filtered amplitude {idx+1}")

for a,b in ax:
    a.set_xlabel("Frequency [Hz]")
    b.set_xlabel("Frequency [Hz]")
    a.set_ylabel("Amplitude")
plt.tight_layout()

# 그림 4-7
savefig("fig/pyssp_melfb")

### 멜스펙트로그램 계산

In [ ]:
# 음성 파일 읽기
sr, x = wavfile.read(ttslearn.util.example_audio_file())
# 5밀리초의 프레임 시프트를 연산합니다.
frame_shift = int(sr * 0.005)
n_fft = 2048

# 스펙트로그램
X = librosa.stft(x.astype(np.float32), n_fft=n_fft, hop_length=frame_shift)

# 80차원 멜스펙트로그램
n_mels = 80
melfb = librosa.filters.mel(sr, n_fft, n_mels=n_mels)
melspec = librosa.amplitude_to_db(np.dot(melfb, np.abs(X)), ref=np.max)

# 비교용 로그 진폭 스펙트로그램
logX = librosa.amplitude_to_db(np.abs(X), ref=np.max)

fig, ax = plt.subplots(2, 1, figsize=(8,6))
ax[0].set_title("Spectrogram")
ax[1].set_title("80-dim Mel-spectrogram")
mesh = librosa.display.specshow(logX, hop_length=frame_shift, sr=sr, cmap=cmap, x_axis="time", y_axis="hz", ax=ax[0])
fig.colorbar(mesh, ax=ax[0], format="%+2.f dB")
mesh.set_clim(-80, 0)
mesh = librosa.display.specshow(melspec, hop_length=frame_shift, sr=sr, cmap=cmap, x_axis="time", y_axis="mel",ax=ax[1])
fig.colorbar(mesh, ax=ax[1], format="%+2.f dB")
mesh.set_clim(-80, 0)

for a in ax:
    a.set_ylim(0, 8000)
    a.set_xlabel("Time [sec]")
    a.set_ylabel("Frequency [Hz]")
plt.tight_layout()

# 그림 4-8
savefig("fig/pyssp_melspectrogram")

## 4.6 Griffin-Lim 알고리즘 기반의 위상 복원

In [ ]:
# 음성 파일 읽기
sr, x = wavfile.read(ttslearn.util.example_audio_file())
# 5밀리초의 프레임 시프트를 연산합니다.
frame_shift = int(sr * 0.005)
n_fft = 2048

# 진폭 스펙트로그램
X = np.abs(librosa.stft(x.astype(np.float32), n_fft=n_fft, hop_length=frame_shift))

y1 = librosa.griffinlim(X, hop_length=frame_shift, n_iter=1)
y2 = librosa.griffinlim(X, hop_length=frame_shift, n_iter=100)

# 오디오 플레이어의 표시
IPython.display.display(Audio(y1, rate=sr))
IPython.display.display(Audio(y2, rate=sr))
IPython.display.display(Audio(x, rate=sr))

fig, ax = plt.subplots(3, 1, figsize=(8,6), sharey=True)
ax[0].set_title("Griffin-Lim # of iteration: 1")
ax[1].set_title("Griffin-Lim # of iteration: 100")
ax[2].set_title("Natural speech")
librosa.display.waveshow(y1, sr=sr, ax=ax[0])
librosa.display.waveshow(y2, sr=sr, ax=ax[1])
librosa.display.waveshow(x.astype(np.float32), sr=sr, ax=ax[2])

for a in ax:
    a.set_xlabel("Time [sec]")
    a.set_ylabel("Amplitude")
plt.tight_layout()

# 그림 4-9
savefig("fig/pyssp_griffin_lim")

### 순간 주파수 가시화(시각화) (bonus)

Griffin-Lim 알고리즘은 위상 복원 기법입니다. 합성 음성과 자연 음성의 순시 위상(위상의 시간 미분)을 비교함으로써 위상 복원이 기대대로 이루어지고 있는지를 시각적으로 확인할 수 있습니다.

In [ ]:
n_fft = 1024
hop_length = n_fft // 4
fig, ax = plt.subplots(1, 3, figsize=(10,5), sharex=True)

C = librosa.stft(y1, n_fft=n_fft, hop_length=hop_length)
ifreq = np.angle(C[:, 1:] * np.conjugate(C[:, :-1]))
mesh = librosa.display.specshow(ifreq, cmap=cmap, ax=ax[0], x_axis="time", y_axis="hz", sr=sr, hop_length=hop_length)
fig.colorbar(mesh, ax=ax[0])
ax[0].set_title("GL # of iteration: 1")

C = librosa.stft(y2, n_fft=n_fft, hop_length=hop_length)
ifreq = np.angle(C[:, 1:] * np.conjugate(C[:, :-1]))
mesh = librosa.display.specshow(ifreq, cmap=cmap, ax=ax[1], x_axis="time", y_axis="hz", sr=sr, hop_length=hop_length)
fig.colorbar(mesh, ax=ax[1])
ax[1].set_title("GL # of iteration: 100")

C = librosa.stft(x.astype(np.float32), n_fft=n_fft, hop_length=hop_length)
ifreq = np.angle(C[:, 1:] * np.conjugate(C[:, :-1]))
mesh = librosa.display.specshow(ifreq, cmap=cmap, ax=ax[2], x_axis="time", y_axis="hz", sr=sr, hop_length=hop_length)
fig.colorbar(mesh, ax=ax[2])
ax[2].set_title("Natural speech")

for a in ax:
    # 나중에 라벨을 다시 붙일 테니 여기에서는 지워둔다
    a.set_ylabel("")

ax[0].set_ylabel("Frequency [Hz]")
for a in ax:
    a.set_xlim(1.5, 3.0)
    a.set_ylim(0, 4000)
    a.set_xlabel("Time [sec]")
    a.set_xticks([1.5, 2.0, 2.5, 3.0])
    
plt.tight_layout()